In [ ]:
import random

class Character:
    """
    Class to represent an individual character for using within our story.
    """
    def __init__(self, string_input):
        ## name ##
        self.name = string_input[0]

        ## attributes ##
        # valid attribute metrics
        self.min_attr_score = 1
        self.max_attr_score = 4
        self.required_sum = 7

        # move input into dictionary
        self.attributes_input = string_input[1]
        attributes = self.attributes_input.split(' ')
        self.attribute_scores  = {}
        for val in attributes:
            if val[0] == "A":
                key = "acumen"
            elif val[0] == "B":
                key = "body"
            elif val[0] == "C":
                key = "charm"
            self.attribute_scores[key] = int(val[1:])
        # perform checks on the input 
        for key, attribute_score in self.attribute_scores.items():
            # check each attribute score is valid
            try:
                assert 1 <= attribute_score <= 4, f"invalid value for {key}; {attribute_score} is not in the range {self.min_attr_score} to {self.max_attr_score}"
            except AssertionError as e:
                raise ValueError(e)
        # check all attribute scores sum to required amount
        try:
            assert sum(self.attribute_scores.values()) == self.required_sum, f"{self.attributes_input} is invalid, sum of attributes does not equal {self.required_sum}"
        except AssertionError as e:
            raise ValueError(e)

        ## skills ##
        # define list of tuples of valid skills
        self.valid_skills = [
            ("Di", "diplomacy", "charm"),
            ("In", "investigation", "acumen"),
            ("Me", "medicine", "acumen"),
            ("La", "language", "charm"),
            ("Ac", "acrobatics", "body"),
            ("Cr", "craft", "body")
        ]
        # split input on spaces
        self.skills_input = string_input[2]
        skills = self.skills_input.split(' ')
        # check that there is a proficiency *
        try:
            assert sum('*' == skill[-1] for skill in skills) == 1, f"{self.skills_input} is invalid; exactly one proficiency asterisk expected"
        except AssertionError as e:
            raise ValueError(e)
        # parse input into skills dictionary
        self.skills = {}
        for skill in skills:
            # check whether proficient or not
            if len(skill) > 2: # asterisk or invalid value present
                skill_name = skill[:2]
                proficiency_boost = 2
            else:
                skill_name = skill
                proficiency_boost = 0
            # check that skill name is a valid name
            try:
                assert skill_name in (skill[0] for skill in self.valid_skills), f"{self.skills_input} is invalid; unexpected skill name given"
            except AssertionError as e:
                raise ValueError(e)
            # find index of skill in our valid_skills
            skill_idx = [skill[0] for skill in self.valid_skills].index(skill_name)
            # get the attribute key e.g. 'acumen'
            attribute_key = self.valid_skills[skill_idx][2]
            # calculate the score based off the attribute and proficiency
            skill_score = self.attribute_scores[attribute_key] + proficiency_boost
            # get the skill key e.g. 'language'
            skill_key = self.valid_skills[skill_idx][1]
            # store the skill score in the dictionary
            self.skills[skill_key] = skill_score

            # check if proficient in this skill and if so store reference
            if proficiency_boost > 0:
                self.proficient_key = skill_key
        
        # create attribute and skill dictionary for reference later
        self.attribute_skills = {**self.attribute_scores, **self.skills} # merges 2 dictionaries
            
    # getters 
    def get_acumen(self):
        return self.attribute_scores["acumen"]
    
    def get_body(self):
        return self.attribute_scores["body"]
    
    def get_charm(self):
        return self.attribute_scores["charm"]
    
    def get_name(self):
        return self.name

    def get_proficient(self):
        return self.proficient_key

    # other methods
    def make_check(self, skill_or_attribute_name, difficulty, override_random):
        # choose whether random number is overridden or a random integer
        if override_random is not None:
            rand_num = override_random
        else:
            rand_num = random.randint(-1,1)
        # calculate final score
        score = self.attribute_skills[skill_or_attribute_name] + rand_num
        # compare score to difficulty
        if score - difficulty >= 3:
            return "++"
        elif score - difficulty >= 0:
            return "+"
        elif score - difficulty >= -3:
            return "-"
        else:
            return "--"


    def __str__(self):
        return f"{self.name} [{self.attributes_input}] is proficient in {self.proficient_key}"
        


class Story:
    """
    Class to represent a story that contains all the data of the relevant scenes, options and appropriate
    characters. Keeps track of the current scene enad allows the choosing of options to progress to the
    next scene.
    """
    def __init__(self, scene_string_data, characters_in_story):
        # values for error checking
        self.valid_attr_skill_keys = ['acumen','body','charm', 'diplomacy', 'investigation', 'medicine', 'language', 'acrobatics', 'craft']

        # initialise the scenes
        # use iterator as easier to work with the specifications
        scenes_iter = iter(scene_string_data)
        self.scenes = []
        while True:
            try:
                # catch ---- at the start of a scene
                line = next(scenes_iter)
                if line == "----":
                    # process an individual scene and add it to the internal list of scenes
                    scene = self.process_individual_scene(scenes_iter) 
                    self.scenes.append(scene)
            except StopIteration:
                break
        # set the active scene to the starting scene for now
        self.active_scene_id = self.scenes[0]["id"]

        # initialise the characters
        # use iterator as easier to work with the specifications
        chars_iter = iter(characters_in_story)
        self.characters = []
        while True:
            try:
                # get name, attributes and skill
                name = next(chars_iter)
                attributes = next(chars_iter)
                skills = next(chars_iter)
                # make list of each of these items
                char_list = [name, attributes, skills]
                # create character and append to list of characters
                character = Character(char_list)
                self.characters.append(character)

                # discard ----
                _ = next(chars_iter)  
            except StopIteration:
                break

    def process_individual_scene(self, scene_iter):
        """Method to process an individual scene, starting from the scene ID.
        This runs through the iterator passed as `scene_iter` and gets the ID,
        description and options for that scene. 

        :param scene_iter: iterator of list of lines that describe a scene
        :type scene_iter: list
        :return: dictionary representing a scene
        """
        scene = {} # initialise empty dictionary
        try:
            # scene ID
            scene["id"] = next(scene_iter)
            # scene description
            line = next(scene_iter)
            scene_description = []
            # iterate until reach ====, which indicates description is over
            while line != "====":
                scene_description.append(line)
                line = next(scene_iter)
            scene["description"] = scene_description
            # scene options for progressing in the story
            options = [] #  empty list of options to fill
            selection_str = next(scene_iter)
            # iterate until ---- signalling end of scene
            while selection_str != "----":
                option = {} # single option represented by a dictionary
                # selection options
                selection_split = selection_str.split(' ')
                # check whether attribute/skill is given
                if selection_split[1][0] == "[":
                    # get the key of that attribute
                    option["attr_skill_key"] = selection_split[1][1:]
                    try:
                        # attempt to coerce the skill difficulty to an integer
                        option["attr_skill_difficulty"] = int(selection_split[2][:-1])
                        # check that the key is a valid key for attribute or skill
                        assert option["attr_skill_key"] in self.valid_attr_skill_keys, f"{selection_split}"
                    except AssertionError as e:
                        raise ValueError(e)
                    except ValueError:
                        raise ValueError("PLACEHOLDER ERROR STRING FROM VALUE ERROR")
                    # we know that description of this option will start at index 3 of selection_split
                    description_start_idx = 3
                else:
                    # define these as None since no attribute/skill present
                    option["attr_skill_key"] = None
                    option["attr_skill_difficulty"] = None
                    # we know that description of this option will start at index 1 of selection_split
                    description_start_idx = 1
                # iterate through the split until we find a + or a - indicating the outcomes are beginning
                for i, word in enumerate(selection_split):
                    if word[0] == '-' or word[0] == '+':
                        description_end_idx = i
                        break
                # join the description again with spaces based on indices found
                option["description"] = ' '.join(selection_split[description_start_idx:description_end_idx])
                
                # populate outcomes for the option. initialise with None in case not given
                option["very_bad_outcome"], option["bad_outcome"] = None, None, 
                option["good_outcome"], option["very_good_outcome"] = None, None
                # go through the remaining few items looking for all different outcomes
                for outcome in selection_split[description_end_idx:]:
                    # logic checks for what outcome type it is
                    if outcome[0] == "-":
                        if outcome[1] == "-":
                            option["very_bad_outcome"] = outcome[2:]
                        else:
                            option["bad_outcome"] = outcome[1:]
                    elif outcome[0] == "+":
                        if outcome[1] == "+":
                            option["very_good_outcome"] = outcome[2:]
                        else:
                            option["good_outcome"] = outcome[1:]
                # append option to the options list
                options.append(option)
                # get next option line string
                selection_str = next(scene_iter)
            # add the options into the scene dictionary and return it
            scene["options"] = options
            return scene
        except StopIteration:
            return scene

    def get_scene_id(self):
        return self.active_scene_id

    def get_scene_from_list_by_id(self, scene_id):
        """Get scene from the list of scenes in the story by ID

        :param scene_id: scene id to search for in the scenes list
        :return: scene represented with ID
        """
        for scene in self.scenes:
            if scene_id == scene["id"]:
                return scene

    def get_active_scene(self):
        """Function to return the active scene"""
        return self.get_scene_from_list_by_id(self.active_scene_id)

    def show_current_scene(self):
        """Method to produce a human readable version of the story with options 
        for them to choose from to progress the story
        """
        # get the active scene and print id
        scene = self.get_active_scene()
        scene_str = f'Scene {scene["id"]}\n'
        # print the scene description
        for line in scene["description"]:
            scene_str += line + '\n'
        scene_str += "====\n"
        # print the options and their descriptions
        for i, option in enumerate(scene["options"]):
            scene_str += f'{i+1}. {option["description"]}\n'
        scene_str += '----'
        return scene_str
    
    def select_option(self, option_number, override):
        """Method to update the active scene when a reader picks which option
        they would like to take at a given intersection in the story.
        Gets the options from the active scene, checks what the outcome is for a 
        given character and then updates the active scene based on that outcome.

        :param option_number: option that the reader wants to take when progressing the story
        :param override: number to override when checking the character's score
        :raises StopIteration: when the game is over (End scene denoted by E in the ID)
        """
        # get the active scene and chosen option
        scene = self.get_active_scene()
        if scene["id"][0] == "E":
            raise StopIteration("the game is over")
        option = scene["options"][option_number-1]
        # get the result for the option for given character
        # catch where the attribute skill key is None
        try:
            result = self.characters[0].make_check(option["attr_skill_key"], option["attr_skill_difficulty"], override)
        except KeyError:
            result = "--" # this will cause rest of function to search upwards from -- to find lowest available outcome

        # shorten the options for use in lists below
        vb, b = option["very_bad_outcome"], option["bad_outcome"]
        g, vg = option["good_outcome"], option["very_good_outcome"]

        # compare the result to known outcomes
        # check if the outcome is None, if so assign outcomes to list 
        # that is then used to iterate through to find another scene to
        if result == "++":
            if option["very_good_outcome"] == None:
                outcomes = [vg, g, b, vb]
            else:
                self.active_scene_id = option["very_good_outcome"]
                return
        elif result == "+":
            if option["good_outcome"] == None:
                outcomes = [g, b, vb, vg]
            else:
                self.active_scene_id = option["good_outcome"]
                return
        elif result == "-":
            if option["bad_outcome"] == None:
                outcomes = [b, vb, g, vg]
            else:
                self.active_scene_id = option["bad_outcome"]
                return
        else:
            if option["very_bad_outcome"] == None:
                outcomes = [vb, b, g, vg]
            else:
                self.active_scene_id = option["very_bad_outcome"]
                return

        # the result did not match a valid outcome -> iterate through other outcomes
        # follows rules of iterate down (i.e. + -> -) and then worst case iterate up (e.g. -- -> ++)
        for outcome in outcomes:
            if outcome != None:
                self.active_scene_id = outcome
                return

    def __str__(self):
        story_str = "CHARACTERS\n"
        for character in self.characters:
            story_str += str(character) + '\n'
        
        story_str += "SCENES\n"
        for scene_idx, scene in enumerate(self.scenes):
            story_str += f"{scene['id']} >"
            for i, option in enumerate(scene["options"]):
                story_str += f" [{i+1}."
                if option['attr_skill_key']:
                   story_str += f" {option['attr_skill_key']}{option['attr_skill_difficulty']}"
                if option['very_good_outcome']:
                    story_str += f" ++{option['very_good_outcome']}"
                if option['good_outcome']:
                    story_str += f" +{option['good_outcome']}"
                if option['bad_outcome']:
                    story_str += f" -{option['bad_outcome']}"
                if option['very_bad_outcome']:
                    story_str += f" --{option['very_bad_outcome']}"
                story_str += "]"
            if (scene_idx + 1) != len(self.scenes):
                story_str += '\n'

        return story_str

if __name__ == "__main__":
    # Character class testing
    test_char = Character(['Val Idoty', 'A2 B2 C3', 'Di* In Me La Ac Cr'])
    print(test_char)

    # testing make_check
    max_power = Character(["Max Power", "A1 B2 C4", "Di In Me La Ac Cr*"])
    assert max_power.make_check("diplomacy", 5, 0) == "-", f'{max_power.make_check("diplomacy", 5, 0)}'
    assert max_power.make_check("diplomacy", 5, 1) == "+", f'{max_power.make_check("diplomacy", 5, 1)}'
    assert max_power.make_check("diplomacy", 5, 4) == "++", f'{max_power.make_check("diplomacy", 5, 4)}'
    assert max_power.make_check("diplomacy", 5, -3) == "--", f'{max_power.make_check("diplomacy", 5, -3)}'


    # Story class testing
    char_text = ["Hero","A2 B3 C2","Di In Me La Ac* Cr","----","doctorb","A3 B3 C1","Di In Me* La Ac Cr"]
    story_text = [
        "----",
        "S",
        "the friends are sitting in the little chicken café together after happily having submitted their assignment 3. This is the most convenient spot for them and was where they worked on the assignment together. Feeling their caffeine levels dropping below optimal, someone heads to the counter and offers to buy everyone a coffee. Many seconds pass while waiting in line (at least seven!) before they reach the front only to discover they left their wallet at home. They decide to...",
        "====",
        "1. [diplomacy 5] use their diplomacy skills to request ask for a freebie -1 +2",
        "2. [acumen 4] draw on all their internal acumen to *will* a coffee into existence ++3 +4 -1",
        "3. [acrobatics 3] use their acrobatics skills to dash home and return with their wallet before the other patrons are the wiser +5 -E~1",
        "4. give up and return to the table -E~1",
        "----","","----",
        "3",
        "Focusing very closely on their desire to be heavily caffeinated",
        "visualising the coffee before them and drawing on all the mental faculties and strength of will they can muster they ",
        'speak their desire to the universe "Oh great provisioners of stimulant beans and their associated beverages, I call upon thee. Share with me your bounty!" They feel a growing solid form of cup within there hand. Minutes pass as they continue to start at their cupped hands and reiterating their desire. The distinct aroma of fresh-brewed coffee wafts its way into', "their nostrils, they feel the warmth between their hands and low and behold in front of their eyes, their will is actioned. They returns to the table with their new coffee and a grin on their face. The friends give a questioning look", '"Something wrong with their machine?". Consequently...',
        '====',
        "1. [language 2] They draws upon their language skills to determine whether the empty hands represents an element of a signlanguage they are familiar with +6 -7 ++8",
        "2. [acrobatics 3] They use their acrobatics skills to get to the counter and order for themselves before they close -E~1 +9",
        '----'
    ]
    st = Story(story_text, char_text)
    print(st)


    # testing show_current_scene
    story_text = [
        "----",
        "28",
        "a group of friends are sitting in the little chicken café together after happily having submitted their assignment 3. This is the most convenient spot for them and was where they worked on the assignment together. Feeling their caffeine levels dropping below optimal, one of you heads to the counter and offers to buy everyone a coffee. Many seconds pass while waiting in line (at least seven!) before you reach the front only to discover you left their wallet at home. You decide to...",
        "====",
        "1. [diplomacy 5] use their diplomacy skills to request ask for a freebie -1 +2",
        "2. [acumen 4] draw on all their internal acumen to *will* a coffee into existence ++3 +4 -1",
        "3. [acrobatics 3] use their acrobatics skills to dash home and return with their wallet before the other patrons are the wiser +5 -E~1",
        "4. give up and return to the table -E~1",
        "----"
    ]
    st = Story(story_text, char_text)
    print(st.show_current_scene())


    # testing selection_option
    story_text = [
        "----",
        "5",
        "nothing important",
        "====",
        "1. [language 4] no issue ++8 +6 -7",
        "2. [charm 3] no issue +9 -E~1",
        "3. [craft 4] no issue ++8 +6 -7",
        "4. no issue -E~1",
        "----"
    ]
    char_text = ["Max Power", "A1 B2 C4", "Di In Me La Ac Cr*"]
    st = Story(story_text, char_text)
    print(st)
    st.select_option(2,-2)
    print(f"(2,-2): {st.active_scene_id}")
    st.active_scene_id = "5"
    st.select_option(1,3)
    print(f"(1,3): {st.active_scene_id}")
    st.active_scene_id = "5"
    st.select_option(3,0)
    print(f"(3,0): {st.active_scene_id}")
    st.active_scene_id = "5"
    st.select_option(2,4)
    print(f"(2,4): {st.active_scene_id}")
    st.active_scene_id = "5"
    st.select_option(1,-4)
    print(f"(1,-4): {st.active_scene_id}")
    st.active_scene_id = "5"
    st.select_option(4,None)
    print(f"(4,None): {st.active_scene_id}")
    st.active_scene_id = "5"